## Import Pandas package

In [4]:
import pyspark.pandas as ps

# Silver layer 
## Combine UPC files into 1 data frame.  Create temp SQL view to run SQL.


In [5]:
path = "abfss://fscustransdevnorthcent@stcustransdevnorthcent.dfs.core.windows.net/bronze/UPC/*/*/*/*.json"
sdfUPCBronzeData_raw = spark.read.option("Multiline","true").json(path)
sdfUPCBronzeData_raw.createOrReplaceTempView("vwUPCBronzeData_Raw")

## Store data in object.  explode rows into individual roecords with column headings

In [6]:
sqlQuery = """
WITH cteUPCBronzeData_Raw AS (
    SELECT explode(body) as rowdata 
    FROM vwUPCBronzeData_Raw
)

SELECT 
     rowdata["Policy Number"] AS PolicyNumber
    ,rowdata["Insured"] AS Insured
    ,rowdata["Description "] AS Description
    ,rowdata["Date Sent"] AS DateSent        
FROM cteUPCBronzeData_Raw"""

## Convert from pandas to spark data frame

In [7]:
sdfUPCBronzeData = spark.sql(sqlQuery)

### Save contents of data frame into Delta format

In [14]:
path = 'abfss://fscustransdevnorthcent@stcustransdevnorthcent.dfs.core.windows.net/silver/upcsilver_raw'
try:
    mssparkutils.fs.rm(path, recurse=True)
except Exception as e:
    pass

sdfUPCBronzeData.write.format("delta").mode("overwrite").save(path)
